In [92]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from lxml import html
import tqdm
import time
import quandl
import pprint
pip install simplejson
import simplejson
import json

In [4]:
#r = requests.get('https://www.quandl.com/api/v3/datasets/WIKI/FB/data.json?api_key={}'.format(QUANDL_API_KEY))

# Part 1 - Scraping

#### Getting the soup

In [16]:
r = requests.get('https://en.wikipedia.org/wiki/FTSE_100_Index')
r.encoding = 'UTF-8'
soup = BeautifulSoup(r.text)
r.status_code

200

In [62]:
def Company():
    '''
    getting the company names
    '''
    firm = []
    names = [a.text.split('\n') for a in soup.find('table',{'id':'constituents'}).tbody.find_all('tr')]
    for name in names[1:]:
        firm.append(name[1])
    return firm


In [63]:
def Epic():
    '''
    getting the company EPIC
    '''
    thickers = []
    names = [a.text.split('\n') for a in soup.find('table',{'id':'constituents'}).tbody.find_all('tr')]
    for name in names[1:]:
        thickers.append(name[2])
    return thickers


In [65]:
def Industry():
    '''
    getting the company industry they operate in
    '''
    branch = []
    names = [a.text.split('\n') for a in soup.find('table',{'id':'constituents'}).tbody.find_all('tr')]
    for name in names[1:]:
          branch.append(name[3])
    return branch

In [72]:
table = pd.DataFrame(Company())
table['EPIC'] = Epic()
table['Industry'] = Industry()
table.columns = ["Company", "EPIC", "Industry"]
table.head()

,Company,EPIC,Industry
0,3i,III,Financial Services
1,Admiral Group,ADM,Nonlife Insurance
2,Anglo American plc,AAL,Mining
3,Antofagasta,ANTO,Mining
4,Ashtead Group,AHT,Support Services


## Part 2 - API requests

In [208]:
payload = { "cuni": "33921250" }

j = requests.post('https://ies-midterm.soulution.rocks/login', data = payload )

# Not able to provide it with a json string

In [209]:
j.text


'{"data": {}, "message": "Error [400] - invalid JSON payload. Submit a JSON string."}'

In [140]:
data = {'cuni':'33921250'}
data_json = simplejson.dumps(data)
payload = {'json_payload': data_json}
j = requests.post('https://ies-midterm.soulution.rocks/login', data=payload)

In [100]:
payload = {'cuni':'33921250'}

login_route = '/login'
url = 'https://ies-midterm.soulution.rocks/'
    
with requests.Session() as s:
    p = s.post(url, data=payload)
    print(p.text)

{"message":"Missing Authentication Token"}


0               Financial Services
1                Nonlife Insurance
2                           Mining
3                           Mining
4                 Support Services
                  ...             
95                  Personal Goods
96    Gas, Water & Multi-utilities
97       Mobile Telecommunications
98              Retail hospitality
99                           Media
Name: Industry, Length: 100, dtype: object

## Part 3

In [182]:
t = table.groupby('Industry').count()
t

,Company,EPIC
Industry,,
Aerospace & Defence,2,2
Asset Managers,1,1
Automobiles & Parts,1,1
Banks,5,5
Beverages,2,2
Chemicals,2,2
Construction & Materials,1,1
Consumer services,1,1
Electricity,1,1


In [158]:
t = pd.DataFrame(t['Company'])


In [186]:
t.max()

Company    8
EPIC       8
dtype: int64